<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 3. Multilayer Perceptron</font></h1>

<h1><font color="#113D68" size=4>6. Proyecto de clasificación binaria</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Rendimiento del modelo de red neuronal de referencia](#section1)
* [2. Optimizar el rendimiento con procesamiento de datos](#section2)
* [3. Ajuste de capas y neuronas](#section3)
    * [3.1. Evaluar una topología más pequeña](#section3.1)
    * [3.2. Evaluar una topología más grande](#section3.2)
    * [3.3. Trabajo a realizar](#section3.3)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En este tutorial se trabajará en un proyecto de clasificación binaria:
* Cómo diseñar y entrenar una red neuronal.
* Cómo evaluar el rendimiento de un modelo de red neuronal.
* Cómo realizar la preparación de datos para mejorar la habilidad al usar redes neuronales.
* Cómo optimizar la topología y configuración de redes neuronales.

In [1]:
import tensorflow as tf
# Eliminar warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Rendimiento del modelo de red neuronal de referencia</font>

En este problema vamos a utilizar un problema de clasificación binaria como es Sonar en el cual los resultados de Accuracy rondan el 84%.

Creemos un modelo de referencia y un resultado para este problema. Comenzaremos importando todas las clases y funciones que necesitaremos.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el dataset [Sonar](https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks))

In [16]:
# Binary Classification with Sonar Dataset: Baseline
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold


# load dataset
dataframe = pd.read_csv("Datasets/sonar.csv", header=None)
dataset = dataframe.values

# split into input (X) and output (Y) variables
X = dataset[:, 0:60].astype(float)
y = dataset[:, 60]


La variable de salida son de tipo string. Debemos convertirlos en valores enteros 0 y 1. Podemos hacer esto usando la clase `LabelEncoder` a través de `fit()` y `transform()`.

In [17]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
encoded_Y

array([[0.02  , 0.0371, 0.0428, ..., 0.0084, 0.009 , 0.0032],
       [0.0453, 0.0523, 0.0843, ..., 0.0049, 0.0052, 0.0044],
       [0.0262, 0.0582, 0.1099, ..., 0.0164, 0.0095, 0.0078],
       ...,
       [0.0522, 0.0437, 0.018 , ..., 0.0138, 0.0077, 0.0031],
       [0.0303, 0.0353, 0.049 , ..., 0.0079, 0.0036, 0.0048],
       [0.026 , 0.0363, 0.0136, ..., 0.0036, 0.0061, 0.0115]])

Para usar modelos de Keras con scikit-learn, debemos usar el contenedor `KerasClassifier`. También toma argumentos que pasará a la llamada a `fit()` como el número de épocas y el tamaño del batch. 

Comencemos por definir la función que crea nuestro modelo de línea de base. 
1. Tendrá una única capa oculta completamente conectada.
2. Se utilizara la función de activación ReLu. 
3. La capa de salida contiene una sola neurona para hacer predicciones utilizando función de activación Sigmoidal. 
4. Se usará la función de pérdida logarítmica binaria (`binary_crossentropy`). 
6. Utilizar el algoritmo de optimización Adam y Accuracy como métrica.

In [19]:
# baseline model
def create_baseline():
    ???

Ahora es el momento de evaluar este modelo. Pasamos el número de épocas de entrenamiento al `KerasClassifier` y realizarmos una validación cruzada de 10-fold.

In [20]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=50, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=5, shuffle=True)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

 Linea base: 83.76% (9.51%)


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Optimizar el rendimiento con procesamiento de datos</font>

La **estandarización** preserva las distribuciones gaussianas mientras normaliza las tendencias centrales para cada atributo. Para ello utilizamos `StandardScaler` de scikit-learn. 

Es una buena práctica entrenar el procedimiento de estandarización en los datos de entrenamiento dentro de una ejecución de validación cruzada y usar la instancia de estandarización entrenada para preparar el fold de validación no etiquetada. Podemos lograr esto en scikit-learn usando una clase `Pipeline`. 


In [23]:
# Binary Classification with Sonar Dataset: Standardized
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# evaluate baseline model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=5, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Modelo Estandarizado: 87.00% (5.74%)


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Ajuste de capas y neuronas</font>

En esta sección echamos un vistazo a dos experimentos sobre la estructura de la red: hacerla más pequeña y hacerla más grande.

<a id="section3.1"></a>
# <font color="#004D7F" size=5>3.1. Evaluar una topología más pequeña</font>

Podemos forzar un tipo de extracción de características por parte de la red restringiendo el espacio de representación en la primera capa oculta.

En este experimento, tomamos nuestro modelo de línea base con 60 neuronas en la capa oculta y lo reducimos a la mitad a 30. 

In [25]:
def create_smaller():
    ???

# evaluate baseline model with standardized dataset
estimators = []
estimators.append((???))
estimators.append((???))
pipeline = ???
kfold = StratifiedKFold(n_splits=5, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Modelo Estandarizado: 88.02% (6.43%)


<a id="section3.2"></a>
# <font color="#004D7F" size=5>3.2. Evaluar una topología más grande</font>

Una topología de red neuronal con más capas ofrece más oportunidades para que la red extraiga características clave y las recombine de formas útiles no lineales. Nuestra red ahora tiene la topología:
```
    60 inputs -> [60 -> 30] -> 1 output
``` 

In [26]:
def create_larger():
    ???

# evaluate baseline model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(model=create_larger, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=5, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Modelo Estandarizado: 87.45% (7.12%)


<a id="section3.3"></a>
# <font color="#004D7F" size=5>3.3. Trabajo a realizar</font>

Con un mayor ajuste de aspectos como el algoritmo de optimización y el número de épocas de entrenamiento, se espera que sean posibles más mejoras. ¿Cuál es la mejor puntuación que puede lograr en este conjunto de datos?

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>